In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing
from torch import nn
from tensordict.nn import TensorDictModule

from collections import defaultdict

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing

In [ ]:
is_fork = multiprocessing.get_start_method() == "fork"

device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

## Environment Preparation

#### Load unity environment using `mlagents_envs`

In [ ]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment

channel = EngineConfigurationChannel()
env_path = "C:/Users/Pawel/Documents/Unity_Project/warehouse-bot-training/environment_builds/test_env_simplified/Warehouse_Bot.exe"

from torchrl.envs import UnityMLAgentsEnv

unity_env = UnityEnvironment(
  file_name=env_path,
  side_channels=[channel],
  # additional_args=["-batchmode", "-nographics"]
)
channel.set_configuration_parameters(time_scale=2)

#### Transform environment from `mlagents` to `gymnasium`

In [ ]:
import gymnasium as gym

In [ ]:
print(gym.__version__)

In [ ]:
from env_gymnasium_wrapper import UnityGymWrapper 

gymnasium_env = UnityGymWrapper(unity_env)

#### Creating stable_baselines3 model

In [ ]:
from stable_baselines3 import PPO

model = PPO.load("./saved_models/test", gymnasium_env)

In [ ]:
env = model.get_env()
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    # env.render("human")